# IBM Data Science
## Coursera Capstone notebook Week 3
This notebook will be used to analyse location data in Toronto for the capstone project of the IBM Data Science course.

### Part 1

In [2]:
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

#!conda install -c conda-forge geopy --yes
import numpy as np

In [3]:
# create the empty dataframe
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
nbhd = pd.DataFrame(columns=column_names)
nbhd

,PostalCode,Borough,Neighborhood


In [6]:
# use beautiful soup to import the data
from bs4 import BeautifulSoup as bs
source_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
